In [1]:
import pandas as pd
import numpy

In [2]:
import codecs
import os
with codecs.open('customer_data.csv', 'r') as file:
    BLOCKSIZE = 512
    if not os.path.exists('customer_data(utf-8).csv'):
        with codecs.open('customer_data(utf-8).csv', "w", "utf-8") as targetFile:
            while True:
                contents = file.read(BLOCKSIZE)
                if not contents:
                    break
                targetFile.write(contents)

FileNotFoundError: [Errno 2] No such file or directory: 'customer_data.csv'

In [2]:
dtype = {
    '交易id': numpy.str,
    '資料日期': numpy.str,
    '資料時間': numpy.str,
    '餐別帶': numpy.str,
    '縣市別': numpy.str,
    '店舖代號': numpy.uint32,
    '主商圈': numpy.str,
    '品號-品名稱': numpy.str,
    '群號-群名稱': numpy.str,
    '單品名稱': numpy.str,
    '銷售數量': numpy.uint16,
    '銷售單價': numpy.float,
    '交易金額': numpy.float
}
USE_COLUMNS = ['交易id', '資料日期', '資料時間', '餐別帶', '縣市別', '店舖代號', '主商圈', '品號-品名稱',
       '群號-群名稱', '單品名稱', '銷售數量', '銷售單價', '交易金額']
PARSE_DATES = {
    '資料日期與時間': [
        '資料日期',
        '資料時間'
    ]
}

In [3]:
file = pd.read_csv('customer_data(utf-8).csv',
                   index_col=1,
                   nrows=100000,
                   usecols=USE_COLUMNS,
                   dtype=dtype,
                   parse_dates=PARSE_DATES,
        )

In [4]:
file = file.dropna()

In [5]:
file.head()

,資料日期與時間,餐別帶,縣市別,店舖代號,主商圈,品號-品名稱,群號-群名稱,單品名稱,銷售數量,銷售單價,交易金額
交易id,,,,,,,,,,,
01637920171201184556000112534201,2017-12-01 18:45:56,晚餐時間帶,台中市,11148,住宅型,72-家庭雜貨,722-家雜用品,（新）銷售用購物袋１８號袋,1,1.0,1.0
01564720171201184641000113754802,2017-12-01 18:46:41,晚餐時間帶,台中市,3860,住宅型,00-傳統代收,11-信用卡,花旗信用卡,1,0.0,0.0
01614720171201163042000114405402,2017-12-01 16:30:42,下午茶時間帶,台中市,10697,住宅型,29-冷藏飲料,296-冷藏奶茶,飲冰室茶集綠奶茶,1,25.0,25.0
01614720171201072745000114390002,2017-12-01 07:27:45,早餐時間帶,台中市,10697,住宅型,29-冷藏飲料,296-冷藏奶茶,義美奶茶,1,35.0,35.0
01614720171201065809000118172701,2017-12-01 06:58:09,早餐時間帶,台中市,10697,住宅型,28-其他乳品,286-發酵乳,養樂多,1,8.0,8.0


In [9]:
def aggregate(df):
    dic = {}
    for index, row in df.iterrows():
        if index not in dic:
            dic[index] = []
        dic[index].append(str(row['單品名稱']))
    dic = { index : value for index, value in dic.items() if len(value) > 1}
    return dic

In [10]:
purchase_list = aggregate(file)

In [11]:
len(purchase_list)

23292

# Apriori

In [15]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [16]:
te = TransactionEncoder()

In [17]:
te_ary = te.fit(purchase_list.values()).transform(purchase_list.values())
df = pd.DataFrame(te_ary, columns=te.columns_)

In [18]:
df.head()

,一口爆漿餡餅（５入）,一度贊爌肉麵（碗）,一度贊爌肉麵（袋）,一度贊麻辣牛肉麵（碗）,一日水果１００％蘋果汁,一日蔬果１００％紫色蔬果汁,一日蔬果１００％蔬果汁,一番榨生啤酒－５００,一銀信用卡,一顆蛋蛋糕,...,ｅＴａｇ條碼卡,ｅＴａｇ繳費,ｅｚ店到店手續費,ｅｚ簡訊寄件手續費,ｋｉｔｋａｔ提拉米蘇巧克力,ｋｉｔｋａｔ酥脆花生奶油巧克力,ｋｉｔｋａｔ黑巧克力,ｏｒｉｏｎ生啤酒,ｐｉｎｋｙ水蜜桃薄荷糖,ｓｕｐｅｒ冰炫薄荷口香糖
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
def get_support_count_table(table, start, end, step):
    result = {}
    for i in range(start, end, step):
        h = float(i / 1000)
        temp_result = apriori(table, min_support=h, use_colnames=True)
        result[i] = temp_result.shape[0]
    return result

In [20]:
get_support_count_table(df, 2, 10, 2)

{2: 373, 4: 156, 6: 65, 8: 39}

In [21]:
result = apriori(df, min_support=0.002, use_colnames=True)

In [22]:
result.shape

(373, 2)

In [23]:
result.sort_values(by=['support'], ascending=False)

,support,itemsets
237,0.052817,(茶葉蛋（銷售用）)
335,0.046719,(（新）銷售用購物袋１８號袋)
12,0.021470,(代收手續費４)
262,0.017004,(蝦皮取件Ｃ)
319,0.016489,(麥香奶茶ＴＰ３００)
74,0.014900,(台灣自來水)
222,0.014857,(義美奶茶)
2,0.014729,(七星１０毫克硬盒香煙)
6,0.014213,(中華電信)
70,0.013226,(台中二段停)


In [24]:
result.shape[0]

373

In [25]:
result["items_sets_len"] = result["itemsets"].apply(lambda x: len(x))
multiple_item_sets = result[result['items_sets_len'] > 1]
single_item_sets = result[(result['items_sets_len'] == 1) & (result['support'] > 0.001)]

In [26]:
multiple_item_sets

,support,itemsets,items_sets_len
347,0.003178,"(代收手續費１５, 中信外１５)",2
348,0.002104,"(代收手續費４, 中華電信)",2
349,0.003736,"(中華電信, 台灣自來水)",2
350,0.002147,"(代收手續費１５, 合庫代１５)",2
351,0.002104,"(寶物交易代, 代收手續費２５)",2
352,0.011207,"(代收手續費４, 健保費代收)",2
353,0.002276,"(代收手續費４, 台灣自來水)",2
354,0.010606,"(代收手續費４, 國民年金代)",2
355,0.002791,"(通行繳費, 代收手續費５)",2
356,0.003349,"(代收折價卷, 奶香綠茶３３０ＭＬ)",2


In [27]:
single_item_sets

,support,itemsets,items_sets_len
0,0.007600,(七星天藍７毫克硬盒),1
1,0.002233,(七星天藍７毫克軟包),1
2,0.014729,(七星１０毫克硬盒香煙),1
3,0.004294,(七星１０毫克軟包),1
4,0.003178,(中信外１５),1
5,0.004809,(中信有線代),1
6,0.014213,(中華電信),1
7,0.003865,(九易購物),1
8,0.002018,(乳香世家全脂鮮乳２２０ｍｌ),1
9,0.002791,(亞太電信代),1


# Association Rules

In [28]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(result, metric="confidence", min_threshold=0.6)

In [29]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(中信外１５),(代收手續費１５),0.003178,0.009060,0.003178,1.000000,110.369668,0.003149,inf
1,(合庫代１５),(代收手續費１５),0.002147,0.009060,0.002147,1.000000,110.369668,0.002128,inf
2,(寶物交易代),(代收手續費２５),0.002104,0.002448,0.002104,1.000000,408.561404,0.002099,inf
3,(代收手續費２５),(寶物交易代),0.002448,0.002104,0.002104,0.859649,408.561404,0.002099,7.110008
4,(健保費代收),(代收手續費４),0.011207,0.021470,0.011207,1.000000,46.576000,0.010967,inf
5,(國民年金代),(代收手續費４),0.010606,0.021470,0.010606,1.000000,46.576000,0.010379,inf
6,(通行繳費),(代收手續費５),0.002791,0.003951,0.002791,1.000000,253.130435,0.002780,inf
7,(代收手續費５),(通行繳費),0.003951,0.002791,0.002791,0.706522,253.130435,0.002780,3.397897
8,(奶香綠茶３３０ＭＬ),(代收折價卷),0.004122,0.008459,0.003349,0.812500,96.048223,0.003314,5.288217
9,(伊藤園蘋果紅茶),(促銷券０６),0.009447,0.012753,0.009275,0.981818,76.985124,0.009155,54.298566


# Network Analysis

In [30]:
import networkx as nx

In [31]:
Graph = nx.Graph()

In [14]:
def convert(purchase_list):
    def find_edges_in_list(itemsets):
        from itertools import combinations
        result = []
        return combinations(itemsets, 2)
    result = {}
    nodes = set()
    for key, itemsets in purchase_list.items():
        if len(itemsets) > 1:
            for item in itemsets:
                nodes.add(item)
            for edge in find_edges_in_list(itemsets):
                if edge in result:
                    result[edge] += 1
                else:
                    result[edge] = 1
    return (nodes, result)

In [15]:
nodes, edges_dict = convert(purchase_list)

In [18]:
len(edges_dict)

50911

In [19]:
len(nodes)

2993

In [39]:
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool, PanTool, WheelZoomTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4

plot = Plot(plot_width=1440, plot_height=1440,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Graph Interaction Demonstration"

node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("club", "@club")])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool(), PanTool(), WheelZoomTool())

graph_renderer = from_networkx(graph, nx.spring_layout, scale=1.5, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)

In [4]:
import igraph

In [5]:
print(igraph.__version__)

0.7.1


In [6]:
g = igraph.Graph()

In [26]:
for node in nodes:
    g.add_vertex(node)

In [30]:
g.add_edges(edges_dict.keys())

In [28]:
'（新）銷售用購物袋１８號袋' in nodes

True

In [29]:
g.vs.find(name='（新）銷售用購物袋１８號袋')

igraph.Vertex(<igraph.Graph object at 0x00000171BDFFB318>, 5110, {'name': '（新）銷售用購物袋１８號袋'})

In [31]:
print(g)

IGRAPH UN-- 5986 50911 --
+ attr: name (v)
+ edges (vertex names):
瑞穗低脂鮮乳９３０ｍｌ--（新）銷售用購物袋１８號袋, 鮮奶油布蕾--（新）銷售用購物袋１８號袋, 統一大布丁（雞蛋口味）--（新）銷售用購物袋１８號袋,
鮮奶茶--（新）銷售用購物袋１８號袋, 鮮奶油布蕾--瑞穗低脂鮮乳９３０ｍｌ, 統一大布丁（雞蛋口味）--瑞穗低脂鮮乳９３０ｍｌ,
鮮奶茶--瑞穗低脂鮮乳９３０ｍｌ, 統一大布丁（雞蛋口味）--鮮奶油布蕾, 鮮奶茶--鮮奶油布蕾, 鮮奶茶--統一大布丁（雞蛋口味）,
飲冰室茶集綠奶茶--雙薯泥燻雞起司, ＦＩＮ健康補給飲料９７５ｍｌ--沙茶牛肉炒麵, 草莓天使蛋糕--冷泡茶台茶１２號紅茶, 愛之味分解茶--多喝水２Ｌ,
光泉米漿--光泉無加糖鮮豆漿, 起司派對三明治--光泉無加糖鮮豆漿, 光泉無加糖鮮豆漿--肉鬆細卷, 光泉米漿--起司派對三明治, 光泉米漿--肉鬆細卷,
起司派對三明治--肉鬆細卷, 波的多洋芋片蚵仔煎口味--卡迪那牛排卡大包, 香烤嫩雞味洋芋片--卡迪那牛排卡大包,
波的多洋芋片蚵仔煎口味--香烤嫩雞味洋芋片, 千島醬--火腿青蔬洋芋沙拉, 千島醬--蕃薯（２０元）, 蕃薯（２０元）--火腿青蔬洋芋沙拉,
新頻道有線--台灣大哥大, 麥香紅茶ＴＰ３００--冷泡茶冰釀烏龍, ＡＷ極酷嗆涼紫冰野莓口香糖--純萃喝濃厚系醇乳奶茶,
韋恩特濃咖啡--雲摩爾ＸＳ香菸－藍, 奶酥麵包--雲摩爾ＸＳ香菸－藍, 韋恩特濃咖啡--奶酥麵包, 泰山仙草蜜--香蕉單入, 鮪魚飯糰--泰山仙草蜜,
鮪魚飯糰--香蕉單入, 立頓奶茶ＴＰ３３０--日式蒜香燒豚飯, 雲絲頓銀３毫克香煙--德州薯條卡大包, 竹北停車費--全家起酥葡萄奶酥,
全家起酥葡萄奶酥--頭份停車費, ＡＢ優酪乳原味--全家起酥葡萄奶酥, 竹北停車費--頭份停車費, 竹北停車費--ＡＢ優酪乳原味,
ＡＢ優酪乳原味--頭份停車費, 台鐵取票--台鐵手續費, 茶葉蛋（銷售用）--簡單點無加糖優酪乳, 帕瑪森起司火腿麵包--簡單點無加糖優酪乳,
茶葉蛋（銷售用）--帕瑪森起司火腿麵包, 熱拿鐵中杯０８６--光泉無加糖黑豆漿, 熱拿鐵中杯０８６--冷泡茶冰釀烏龍, 光泉無加糖黑豆漿--冷泡茶冰釀烏龍,
義美奶茶--墨西哥葡萄奶酥